## <노드 실습>

- 프로젝트 결과 사진 첨부 및 결과 정리는 'MainQuest_9_뉴스 카테고리 다중분류[단어장 10000].ipynb' 에 있습니다.

### 노드 19 - 뉴스 카테고리 다중분류

### 1) 라이브러리/데이터 불러오기/전처리

In [1]:
!pip install gensim


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from tensorflow.keras.datasets import reuters
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#### **단어장 크기: 5000**

In [2]:
# 이부분에있는 num_words를 5000과 다른방법으로 바꿔보세요~

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2) ##

In [3]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

In [4]:
index_to_word = { index+3 : word for word, index in word_index.items() }
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

In [5]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [6]:
decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test
print(len(x_test))

2246


In [7]:
# 벡터화 DTM, TF-idf 방법
dtmvector = CountVectorizer()

tfidf_transformer = TfidfTransformer()

x_train_dtm = dtmvector.fit_transform(x_train)
x_test_dtm= dtmvector.transform(x_test)

x_train_tfidf = tfidf_transformer.fit_transform(x_train_dtm)
x_test_tfidf = tfidf_transformer.transform(x_test_dtm)

In [8]:
x_train[3]

"<sos> the farmers home administration the u s agriculture department's farm lending arm could lose about seven billion dlrs in outstanding principal on its severely <unk> borrowers or about one fourth of its farm loan portfolio the general accounting office gao said in remarks prepared for delivery to the senate agriculture committee brian <unk> senior associate director of gao also said that a preliminary analysis of proposed changes in <unk> financial <unk> standards indicated as many as one half of <unk> borrowers who received new loans from the agency in 1986 would be <unk> under the proposed system the agency has proposed <unk> <unk> credit using a variety of financial <unk> instead of <unk> <unk> on <unk> ability senate agriculture committee chairman <unk> <unk> d <unk> <unk> the proposed <unk> changes telling <unk> administrator <unk> clark at a hearing that they would mark a dramatic shift in the <unk> purpose away from being <unk> <unk> of last <unk> toward becoming a big cit

In [9]:
# 벡터화 W2V방법
from gensim.models import Word2Vec

# 우선 문장을 토큰화 시킵시다 띄어쓰기 기반으로 해볼게요! -> # 위에서 DTM만들때는 왜 안해줬냐! -> CountVectorizer에서 띄어쓰기 기반 토큰화가 내장되있음
x_train_tokenized = [sentence.split() for sentence in x_train]
x_test_tokenized = [sentence.split() for sentence in x_test]

# vector사이즈를 늘리거나 줄여보세요 아마 512 가장많이쓰이는 방식
model = Word2Vec(sentences = x_train_tokenized, vector_size = 256, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


In [10]:
# W2V이 잘되었는지 확인 -> 여차저차 되긴한것같다
model_result = model.wv.most_similar('man')
print(model_result)

[('indiana', 0.8248819708824158), ('smc', 0.8099660873413086), ('gordon', 0.7935830354690552), ('d', 0.7923528552055359), ('rica', 0.7895998358726501), ('lawrence', 0.7895039319992065), ('charles', 0.7792946100234985), ('reed', 0.7734869718551636), ('ontario', 0.7733910083770752), ('aegean', 0.773090660572052)]


In [11]:
# 학습된 Word2Vec 모델
w2v_model = model

# 각 문장을 벡터화 시키는 코드
def vectorize_sentence(sentence, model, max_len):
    vecs = []
    for word in sentence:
        if word in model.wv:
            vecs.append(model.wv[word])
        else:
            vecs.append(np.zeros(model.vector_size))
    # Padding
    if len(vecs) < max_len:
        vecs += [np.zeros(model.vector_size)] * (max_len - len(vecs))
    else:
        vecs = vecs[:max_len]
    return np.array(vecs)


# 최대 문장길이를 잘 잡아주세요
x_train_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_train_tokenized])
x_test_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_test_tokenized])

In [12]:
print(x_train_w2v.shape)
print(x_test_w2v.shape)

(8982, 100, 256)
(2246, 100, 256)


### 2) 머신러닝 모델 학습
- 6/18(수)에 진행했던 모델들을 사용해서 돌려본다..!
- 모델 종류 -> 컴플리먼트 나이브베이즈(CNB), 로지스틱회귀, 선형 SVM, Decision Tree, 랜덤포레스트, 그래디언트 부스팅 트리(GBC), 보팅(Voting), XGBoost

#### 1) 컴플리먼트 나이브베이즈(CNB)
- 로이터 뉴스 분류는 다중 클래스 이므로, average='weighted'로 했다.

In [24]:
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, f1_score

cb = ComplementNB()
cb.fit(x_train_tfidf, y_train)

predicted = cb.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.7707034728406055
F1-score: 0.7458990404916549


#### 2) 로지스틱회귀

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
lr.fit(x_train_tfidf, y_train)

predicted = lr.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.8032056990204809
F1-score: 0.7981072251211082


#### 3) 선형 SVM

In [29]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
lsvc.fit(x_train_tfidf, y_train)

predicted = lsvc.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.7769367764915405
F1-score: 0.7733965094936557


c:\Users\lynn9\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 4) Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(x_train_tfidf, y_train)

predicted = tree.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.6179875333926982
F1-score: 0.5729970881280324


#### 5) 랜덤포레스트

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(x_train_tfidf, y_train)

predicted = forest.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.701246660730187
F1-score: 0.6770217603524399


#### 6) 그래디언트 부스팅 트리 

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(x_train_tfidf, y_train)

predicted = grbt.predict(x_test_tfidf)  # 테스트 데이터 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.7671415850400712
F1-score: 0.7658073253539026


#### 7) 보팅
- soft 보팅 사용했다.

In [33]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

log_clf = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
nb_clf = ComplementNB()
gb_clf = GradientBoostingClassifier()

# Voting Classifier 선언
voting_classifier = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('nb', nb_clf),
        ('gb', gb_clf)
    ],
    voting='soft'
)
voting_classifier.fit(x_train_tfidf, y_train)

predicted = voting_classifier.predict(x_test_tfidf) #테스트 데이터에 대한 예측

# 정확도와 F1-score 출력
print("정확도:", accuracy_score(y_test, predicted))
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

정확도: 0.792520035618878
F1-score: 0.7901691650403507


### 8) XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

xgb = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, use_label_encoder=False, eval_metric='logloss')
xgb.fit(x_train_tfidf, y_train)

predicted = xgb.predict(x_test_tfidf)

# 정확도와 F1-score 출력
print("XGBoost 정확도:", accuracy_score(y_test, predicted))

c:\Users\lynn9\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:158: UserWarning: [18:27:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost 정확도: 0.8018699910952805


In [ ]:
# f1-score 출력
print("F1-score:", f1_score(y_test, predicted, average='weighted'))  # 다중 클래스면 'weighted' 또는 'macro' 사용

F1-score: 0.7947532747821542


### 3) 딥러닝


#### try1 -> Dense NN 모델

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, Dense, Dropout


dense_model = Sequential([
    Flatten(input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요!
])

dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dense_model.summary()

# 시간이 좀 걸립니다! 한 20분정도..
dense_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred_proba = dense_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

c:\Users\lynn9\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    13,107,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 46)             │         5,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,179,310 (50.28 MB)

 Trainable params: 13,179,310 (50.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 111s 423ms/step - accuracy: 0.5456 - loss: 2.0806 - val_accuracy: 0.6739 - val_loss: 1.4422
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 75s 333ms/step - accuracy: 0.7057 - loss: 1.2803 - val_accuracy: 0.6884 - val_loss: 1.3752
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 73s 326ms/step - accuracy: 0.7611 - loss: 0.9888 - val_accuracy: 0.6984 - val_loss: 1.3716
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 83s 370ms/step - accuracy: 0.8170 - loss: 0.7612 - val_accuracy: 0.6989 - val_loss: 1.4517
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 83s 370ms/step - accuracy: 0.8198 - loss: 0.7462 - val_accuracy: 0.6973 - val_loss: 1.4987
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 92s 408ms/step - accuracy: 0.8674 - loss: 0.5891 - val_accuracy: 0.6867 - val_loss: 1.5176
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 122s 318ms/step - accuracy: 0.8767 - loss: 0.5232 - val_accuracy: 0.6962 - val_loss: 1.5621
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 84s 372ms/step - accuracy: 0.8920 - loss:

#### try2 -> RNN 딥러닝 모델

In [27]:
# rnn 시계열 특징 데이터 특화 모델
rnn_model = Sequential([
    LSTM(128, input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요~
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

# 시간이 좀 걸립니다! 한 20분정도
rnn_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred_proba = rnn_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

c:\Users\lynn9\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,366 (813.93 KB)

 Trainable params: 208,366 (813.93 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 77s 285ms/step - accuracy: 0.3762 - loss: 2.6443 - val_accuracy: 0.5743 - val_loss: 1.7291
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 44s 194ms/step - accuracy: 0.5987 - loss: 1.6956 - val_accuracy: 0.6578 - val_loss: 1.4401
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 39s 172ms/step - accuracy: 0.6422 - loss: 1.4675 - val_accuracy: 0.6728 - val_loss: 1.3755
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 59s 261ms/step - accuracy: 0.6608 - loss: 1.4029 - val_accuracy: 0.6917 - val_loss: 1.2814
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 62s 277ms/step - accuracy: 0.6767 - loss: 1.3412 - val_accuracy: 0.6989 - val_loss: 1.2724
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 61s 269ms/step - accuracy: 0.6935 - loss: 1.2771 - val_accuracy: 0.6884 - val_loss: 1.2110
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 236ms/step - accuracy: 0.7094 - loss: 1.1526 - val_accuracy: 0.7101 - val_loss: 1.1949
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 46s 199ms/step - accuracy: 0.7342 - loss: 1

#### try 3 -> 양방향 LSTM

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# 양방향 LSTM 모델 구성
rnn_model = Sequential([
    Bidirectional(LSTM(128), input_shape=(100, 256)),  # 양방향 적용
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')  # 클래스 수 = 46
])

# 컴파일
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

# 학습 (시간 다소 소요)
rnn_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

# 예측
y_pred_proba = rnn_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

# 평가
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

c:\Users\lynn9\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 413,678 (1.58 MB)

 Trainable params: 413,678 (1.58 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 98s 331ms/step - accuracy: 0.5104 - loss: 2.1105 - val_accuracy: 0.6973 - val_loss: 1.2744
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 65s 291ms/step - accuracy: 0.6960 - loss: 1.3103 - val_accuracy: 0.7329 - val_loss: 1.1370
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 75s 257ms/step - accuracy: 0.7375 - loss: 1.0986 - val_accuracy: 0.7396 - val_loss: 1.0661
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 89s 288ms/step - accuracy: 0.7509 - loss: 1.0344 - val_accuracy: 0.7596 - val_loss: 1.0158
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 57s 252ms/step - accuracy: 0.7692 - loss: 0.9357 - val_accuracy: 0.7568 - val_loss: 1.0086
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 58s 256ms/step - accuracy: 0.7921 - loss: 0.8382 - val_accuracy: 0.7830 - val_loss: 0.9376
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 234ms/step - accuracy: 0.8018 - loss: 0.7940 - val_accuracy: 0.7863 - val_loss: 0.9274
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 64s 284ms/step - accuracy: 0.8235 - loss: 0